In [1]:
# pip install jupyterthemes
from jupyterthemes import get_themes
from jupyterthemes.stylefx import set_nb_theme
themes = get_themes()
set_nb_theme(themes[3])

In [2]:
# 1. magic to print version
# 2. magic so that the notebook will reload external python modules
# 3. to use cython
%load_ext watermark
%load_ext autoreload 
%autoreload 2
%load_ext cython

import numpy as np
from numba import jit

%watermark -a 'Ethen' -d -t -v -p numpy,cython,numba

Ethen 2016-12-22 10:28:45 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.2
cython 0.24
numba 0.26.0


# High Performance Python

## Cython

Simplicity of Python and efficiency of C.

We can write it in notebooks by loading the cython magic.

In [3]:
%%cython
def hello_snippet():
    """
    after loading the cython magic, we can
    run the cython code (this code isn't 
    different from normal python code)
    """
    print('hello cython')

In [4]:
hello_snippet()

hello cython


Or write it as a .pyx file and use `setup.py` to compile it:

helloworld.pyx
```python
# cython hello world
def hello():
	print('Hello, World!')
```

setup.py
```python
# compiling the .pyx module
from distutils.core import setup
from Cython.Build import cythonize

# key-value pairs that tell disutils the name
# of the application and which extensions it
# needs to build, for the cython modules, we
# use glob patterns, e.g. *.pyx or simply pass in
# the filename.pyx
setup(
	name = 'Hello',
	ext_modules = cythonize('*.pyx'),
)

# after that run 
# python setup.py build_ext --inplace
# in the command line, and we can import it like
# normal python modules
```

In [5]:
from helloworld import hello
hello()

Hello, World!


### Static Typing

Cython extends the Python language with static type declarations. This increases speed by not needing to do type-checks when running the program. The way we do this in Cython is by adding the `cdef` keyword.

We'll write a simple program that increments j by 1 for 1000 times and compare the speed difference when adding the type declaration.

In [6]:
%%cython

def example():
    """simply increment j by 1 for 1000 times"""
    # declare the integer type before using it
    cdef int i, j = 0
    for i in range(1000):
        j += 1
    
    return j

In [7]:
def example_py():
    j = 0
    for i in range(1000):
        j += 1
    
    return j

In [8]:
%%timeit 
example()

The slowest run took 201.26 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 58.6 ns per loop


In [9]:
%%timeit
example_py()

10000 loops, best of 3: 59.2 µs per loop


Notice the runtime difference (look at the units)

### Functions

To declare functions we use the `cpdef` keyword.

In [10]:
%%cython

cpdef int compute_sum(int a, int b):
    return a + b

In [11]:
compute_sum(5, 3)

8

Notice apart from declaring the function using the `cpdef` keyword, we also specify the return type to be a integer and a two input argument to be integers.

There's still an overhead to calling functions, so if the function is small and is in a computationally expensive for loop, then we can add the `inline` keyword in the function declaration. By doing this, it will replace function call solely with the function body, thus reducing the time to call the function multiple times.

In [12]:
%%cython

cpdef inline int compute_sum(int a, int b):
    return a + b

### Numpy

Typed memoryviews allow even more efficient numpy manipulation since again, it does not incur the python overhead.

In [13]:
%%cython

import numpy as np

# declare memoryviews by using : in the []
cdef double[:, :] b = np.zeros((3, 3), dtype = 'float64')
b[1] = 1

# it now prints memoryview instead of original numpy array
print(b[0])

<MemoryView of 'ndarray' object>


## Pairwise Distance Example

We'll start with simple version of the function that will give us a good benchmark for comparison with Cython alternatives below.

In [14]:
import numpy as np

def euclidean_distance(x1, x2):
    dist = np.sqrt( np.sum( (x1 - x2) ** 2 ) )
    return dist

def pairwise(X, metric = 'euclidean'):
    if metric == 'euclidean':
        dist_func = euclidean_distance
    else:
        raise ValueError("unrecognized metric")    

    n_samples = X.shape[0]
    D = np.zeros((n_samples, n_samples))
    
    # We could exploit symmetry to reduce the number of computations required,
    # i.e. distance D[i, j] = D[j, i]
    # by only looping over its upper triangle
    # but we'll skip that step for now: 
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            dist = dist_func(X[i], X[j])
            D[i, j] = dist
            D[j, i] = dist

    return D

In [15]:
X = np.random.random((1000, 3))
%timeit pairwise(X)

1 loop, best of 3: 3.06 s per loop


We'll try re-writing this into Cython using type memoryview. The key thing with Cython is to avoid using Python objects and function calls as much as possible, including vectorized operations on numpy arrays. This usually means writing out all of the loops by hand and operating on single array elements at a time. 

All the commented `.pyx` code can be found in the [github folder](https://github.com/ethen8181/machine-learning/tree/master/python/cython).

In [16]:
# pairwise1.pyx
from pairwise1 import pairwise1

# test optimized code on a larger matrix
X = np.random.random((4000, 3))
%timeit pairwise1(X)

1 loop, best of 3: 398 ms per loop


We can see the huge speedup over the pure python version! It turns out, though, that we can do even better. If we look in the code, the slicing operation when we call X[i] and X[j] must generate a new numpy array each time. So this time, we will directly slice the X array without  creating new array each time.

In [17]:
from pairwise2 import pairwise2
%timeit pairwise2(X)

10 loops, best of 3: 138 ms per loop


We now try utilize Cython's parallel functionality. (For some reason can't compile the parallel version, will come back to this in the future)

In [18]:
from pairwise3 import pairwise3
%timeit pairwise3(X)

1 loop, best of 3: 265 ms per loop


## Numba

Numba is an LLVM compiler for python code, which allows code written in Python to be converted to highly efficient compiled code in real-time. To use it, we simply add a `@jit` (just in time compilation) decorator to our function.

In [19]:
@jit
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.zeros((M, M), dtype = np.float64)
    for i in range(M):
        for j in range(i + 1, M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            
            dist = np.sqrt(d)
            D[i, j] = dist
            D[j, i] = dist
    
    return D

In [20]:
%timeit pairwise_python(X)

1 loop, best of 3: 154 ms per loop


## Reference

- [Blog: Memoryview Benchmarks](https://jakevdp.github.io/blog/2012/08/08/memoryview-benchmarks/)
- [Blog: Numba vs. Cython: Take 2](http://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/)
- [Tutorial: C for Python programmers](http://www.toves.org/books/cpy/)
- [StackOverflow: Speeding up distance matrix computation with Numpy and Cython](http://stackoverflow.com/questions/25213603/speeding-up-distance-matrix-computation-with-numpy-and-cython)